In [1]:
import pandas as pd
import numpy as np
import sqlite3
from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD

In [2]:
connR = sqlite3.connect('ratings.sqlite')
curR = connR.cursor()
curR.execute('SELECT user_id, book_id, rating FROM Ratings')
resultR = curR.fetchall()
columnR = ["user_id", "book_id", "rating"]
ratings = pd.DataFrame(resultR, columns=columnR)
ratings

,user_id,book_id,rating
0,276725,034545104X,0.0
1,276726,155061224,2.5
2,276727,446520802,0.0
3,276729,052165615X,1.5
4,276729,521795028,3.0
...,...,...,...
1048570,250764,451410777,0.0
1048571,250764,452264464,4.0
1048572,250764,048623715X,0.0
1048573,250764,486256588,0.0


In [3]:
connB = sqlite3.connect('bookdb.sqlite')
curB = connB.cursor()
curB.execute('SELECT book_id, Title FROM Book')
resultB = curB.fetchall()
columnB = ["book_id", "Title"]
books = pd.DataFrame(resultB, columns=columnB)
books

,book_id,Title
0,195153448,Classical Mythology
1,2005018,Clara Callan
2,60973129,Decision in Normandy
3,374157065,Flu: The Story of the Great Influenza Pandemic...
4,393045218,The Mummies of Urumchi
...,...,...
271374,440400988,There's a Bat in Bunk Five
271375,525447644,From One to One Hundred
271376,006008667X,Lily Dale : The True Story of the Town that Ta...
271377,192126040,Republic (World's Classics)


In [4]:
connU = sqlite3.connect('userdb.sqlite')
curU = connU.cursor()
curU.execute('SELECT user_id, Location, Age FROM User')
resultU = curU.fetchall()
columnU = ["user_id", "Location", "Age"]
users = pd.DataFrame(resultU, columns=columnU)
users

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278854,278855,"portland, oregon, usa",NaN
278855,278856,"tacoma, washington, united kingdom",50.0
278856,278857,"brampton, ontario, canada",NaN
278857,278858,"knoxville, tennessee, usa",NaN


In [5]:
combine_book_rating = pd.merge(ratings, books, on='book_id')
combine_book_rating

,user_id,book_id,rating,Title
0,276725,034545104X,0.0,Flesh Tones: A Novel
1,2313,034545104X,2.5,Flesh Tones: A Novel
2,6543,034545104X,0.0,Flesh Tones: A Novel
3,8680,034545104X,2.5,Flesh Tones: A Novel
4,10314,034545104X,4.5,Flesh Tones: A Novel
...,...,...,...,...
77886,250705,089577447X,0.0,National Parks: Explore America
77887,250709,156201238X,2.5,The Captive's Journey
77888,250709,156931005X,2.5,Ad Police (Ad Police)
77889,250764,002736660X,0.0,Branigan's Dog


In [6]:
combine_book_rating = combine_book_rating.dropna(axis=0, subset = ['Title'])

In [7]:
book_ratingCount = (combine_book_rating.groupby(by= ['Title'])['rating'].count().reset_index().rename(columns = {'rating': 'Total_Rating_Count'})[['Title', 'Total_Rating_Count']])
book_ratingCount

,Title,Total_Rating_Count
0,Earth Prayers From around the World: 365 Pray...,9
1,Flight of Fancy: American Heiresses (Zebra Ba...,2
2,Little Comic Shop of Horrors (Give Yourself G...,2
3,"Q-Zone (Star Trek The Next Generation, Book 48)",9
4,The Town Cats and Other Tales,1
...,...,...
20608,together by christmas,8
20609,voyage fatal,1
20610,why I'm like this : True Stories,11
20611,Ã?Â?ber die Regierung.,1


In [8]:
rating_with_total_rating = combine_book_rating.merge(book_ratingCount, left_on = 'Title', right_on = 'Title', how = 'left')
rating_with_total_rating

,user_id,book_id,rating,Title,Total_Rating_Count
0,276725,034545104X,0.0,Flesh Tones: A Novel,53
1,2313,034545104X,2.5,Flesh Tones: A Novel,53
2,6543,034545104X,0.0,Flesh Tones: A Novel,53
3,8680,034545104X,2.5,Flesh Tones: A Novel,53
4,10314,034545104X,4.5,Flesh Tones: A Novel,53
...,...,...,...,...,...
77886,250705,089577447X,0.0,National Parks: Explore America,1
77887,250709,156201238X,2.5,The Captive's Journey,1
77888,250709,156931005X,2.5,Ad Police (Ad Police),1
77889,250764,002736660X,0.0,Branigan's Dog,1


In [9]:
pd.set_option('display.float_format', lambda x: '%.3F' % x)
print(book_ratingCount['Total_Rating_Count'].describe())

count   20613.000
mean        3.779
std        14.107
min         1.000
25%         1.000
50%         1.000
75%         3.000
max       761.000
Name: Total_Rating_Count, dtype: float64


In [10]:
print(book_ratingCount['Total_Rating_Count']. quantile(np.arange(.9, 1, .01))),

0.900    6.000
0.910    7.000
0.920    7.000
0.930    8.000
0.940    9.000
0.950   11.000
0.960   14.000
0.970   17.000
0.980   24.000
0.990   41.000
Name: Total_Rating_Count, dtype: float64


(None,)

In [11]:
popularity_threshold = 50
rating_popular_book = rating_with_total_rating.query('Total_Rating_Count >- @popularity_threshold')
rating_popular_book

,user_id,book_id,rating,Title,Total_Rating_Count
0,276725,034545104X,0.000,Flesh Tones: A Novel,53
1,2313,034545104X,2.500,Flesh Tones: A Novel,53
2,6543,034545104X,0.000,Flesh Tones: A Novel,53
3,8680,034545104X,2.500,Flesh Tones: A Novel,53
4,10314,034545104X,4.500,Flesh Tones: A Novel,53
...,...,...,...,...,...
77886,250705,089577447X,0.000,National Parks: Explore America,1
77887,250709,156201238X,2.500,The Captive's Journey,1
77888,250709,156931005X,2.500,Ad Police (Ad Police),1
77889,250764,002736660X,0.000,Branigan's Dog,1


In [12]:
combined = rating_popular_book.merge(users, left_on = 'user_id', right_on = 'user_id', how = 'left')
us_canada_user_rating = combined[combined['Location'].str.contains("usa|canada")]
us_canada_user_rating = us_canada_user_rating.drop('Age', axis =1)
us_canada_user_rating

,user_id,book_id,rating,Title,Total_Rating_Count,Location
1,2313,034545104X,2.500,Flesh Tones: A Novel,53,"cincinnati, ohio, usa"
2,6543,034545104X,0.000,Flesh Tones: A Novel,53,"strafford, missouri, usa"
3,8680,034545104X,2.500,Flesh Tones: A Novel,53,"st. charles county, missouri, usa"
4,10314,034545104X,4.500,Flesh Tones: A Novel,53,"beaverton, oregon, usa"
5,23768,034545104X,0.000,Flesh Tones: A Novel,53,"st. louis, missouri, usa"
...,...,...,...,...,...,...
77886,250705,089577447X,0.000,National Parks: Explore America,1,"salt lake city, utah, usa"
77887,250709,156201238X,2.500,The Captive's Journey,1,"chicago, illinois, usa"
77888,250709,156931005X,2.500,Ad Police (Ad Police),1,"chicago, illinois, usa"
77889,250764,002736660X,0.000,Branigan's Dog,1,"cove, oregon, usa"


In [19]:
if not us_canada_user_rating[us_canada_user_rating.duplicated(['user_id', 'Title'])].empty:
    initial_rows = us_canada_user_rating.shape[0]
    
    print('Initial dataframeshape {0}'.format(us_canada_user_rating.shape))
    us_canada_user_rating = us_canada_user_rating.drop_duplicates(['user_id', 'Title'])
    current_rows = us_canada_user_rating.shape[0]
    print('New dataframe shape {0}'.format(us_canada_user_rating.shape))
    print('Removed {0} rows'.format(initial_rows - current_rows))

In [21]:
us_canada_user_rating_pivot = us_canada_user_rating.pivot(index = 'Title', columns = 'user_id', values = 'rating').fillna(0)
us_canada_user_rating_matrix = csr_matrix(us_canada_user_rating_pivot.values)

MemoryError: Unable to allocate 1.82 GiB for an array with shape (16457, 14838) and data type float64

In [22]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(us_canada_user_rating_matrix)

NameError: name 'us_canada_user_rating_matrix' is not defined

In [23]:
query_index = np.random.choice(us_canada_user_rating_pivot.shape[0])
distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index, :].values.reshape(1,-1), n_neighbors = 10)
for i in range(len(distances.flatten())):
    if i == 0:
        print('Recommendation for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

NameError: name 'us_canada_user_rating_pivot' is not defined